# Image Recognition

Il progetto verrà gestito con YOLOv8 e Object365 scaricato localmente con appostito script personalizzato.

## ✅ Step 1 – Preparazione
Assicurarsi che la libreria `ultralytics` sia installata

In [ ]:
%pip install --quiet ultralytics

Il dataset scaricato non ha la cartella di test ma solo `train` e `val` che sono per appunto addestramento e validazione. Ho deciso di creare una cartella di test. 

In [ ]:
import os
import shutil
from pathlib import Path
import random
from concurrent.futures import ThreadPoolExecutor

def move_patch(patch_name, val_images_dir, test_images_dir, val_labels_dir, test_labels_dir):
    """
    Sposta una singola patch di immagini e le relative etichette dalla directory di validazione a quella di test.
    Args:
        patch_name (str): Nome della patch da spostare.
        val_images_dir (Path): Directory delle immagini di validazione.
        test_images_dir (Path): Directory delle immagini di test.
        val_labels_dir (Path): Directory delle etichette di validazione.
        test_labels_dir (Path): Directory delle etichette di test.
    """
    # Sposta le immagini
    src_patch_images = val_images_dir / patch_name
    dst_patch_images = test_images_dir / patch_name
    shutil.move(src_patch_images, dst_patch_images)
    print(f"Spostata cartella immagini: {patch_name}")

    # Sposta le etichette (se esistono)
    src_patch_labels = val_labels_dir / patch_name
    dst_patch_labels = test_labels_dir / patch_name
    if os.path.exists(src_patch_labels):
        shutil.move(src_patch_labels, dst_patch_labels)
        print(f"Spostata cartella labels: {patch_name}")

def create_test_set(base_dir, test_size_ratio=0.1, num_threads=4):
    """
    Crea una cartella 'test' e sposta una frazione delle immagini dalla cartella 'val' in parallelo.
    Assume una struttura base_dir/images/{train, val} e base_dir/labels/{train, val}.

    Args:
        base_dir (str): Percorso della directory principale del dataset.
        test_size_ratio (float): Frazione delle immagini di validation da spostare nel test set.
        num_threads (int): Numero di thread da utilizzare per il multithreading.
    """
    val_images_dir = Path(base_dir) / "images" / "val"
    test_images_dir = Path(base_dir) / "images" / "test"
    val_labels_dir = Path(base_dir) / "labels" / "val"
    test_labels_dir = Path(base_dir) / "labels" / "test"

    # Crea le directory test/images e test/labels se non esistono
    test_images_dir.mkdir(parents=True, exist_ok=True)
    test_labels_dir.mkdir(parents=True, exist_ok=True)

    # Ottieni la lista di tutte le sottocartelle (patch) in val/images
    patch_dirs = [d for d in os.listdir(val_images_dir) if (val_images_dir / d).is_dir()]
    num_patches = len(patch_dirs)
    num_test_patches = int(num_patches * test_size_ratio)
    test_patches = random.sample(patch_dirs, num_test_patches)

    print(f"Spostando {num_test_patches} patch ({test_size_ratio*100}%) da validation a test...")

    # Usa ThreadPoolExecutor per spostare le patch in parallelo
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(
                move_patch,
                patch_name,
                val_images_dir,
                test_images_dir,
                val_labels_dir,
                test_labels_dir,
            )
            for patch_name in test_patches
        ]

        # Attendi il completamento di tutti i task
        for future in futures:
            future.result()

    print("Creazione test set completata.")

# Esegui la creazione del test set
base_dir = "/mnt/d/objects365"  # Assicurati che questo sia il percorso corretto
create_test_set(base_dir, test_size_ratio=0.1, num_threads=8)

Siccome YOLO ha bisogno di un file di configurazione in formato YAML (Yet Another Markup Language) e siccome il dataset è diviso in patch c'è bisogno di dare le coordinate delle patch a YOLO e per farlo torna comodo un file TXT con tutti i percorsi.

In [ ]:
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_patch(patch_dir, base_path, f):
    """
    Processa una singola patch e scrive i percorsi delle immagini nel file di output.
    Args:
        patch_dir (str): Nome della directory della patch.
        base_path (Path): Percorso base delle immagini.
        f (file object): File di output aperto in modalità scrittura.
    """
    patch_path = base_path / patch_dir
    if patch_path.is_dir():
        for img in patch_path.glob("*.jpg"):
            f.write(f"{img.absolute()}\n")
        print(f"Completata patch: {patch_dir}")

def create_image_list(base_dir, split, output_file, num_threads=4):
    """
    Crea un file di testo con l'elenco delle immagini per un determinato split (train, val, test).
    Utilizza il multithreading per processare le patch in parallelo.

    Args:
        base_dir (str): Percorso della directory principale del dataset.
        split (str): Split del dataset (train, val, test).
        output_file (str): Percorso del file di output.
        num_threads (int): Numero di thread da utilizzare per il multithreading.
    """
    base_path = Path(base_dir) / "images" / split
    patch_dirs = [d for d in os.listdir(base_path) if (base_path / d).is_dir()]

    print(f"Inizio creazione lista immagini per '{split}'...")
    print(f"Numero di patch da processare: {len(patch_dirs)}")

    with open(output_file, 'w') as f:
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = {
                executor.submit(process_patch, patch_dir, base_path, f): patch_dir
                for patch_dir in patch_dirs
            }

            for future in as_completed(futures):
                patch_name = futures[future]
                try:
                    future.result()
                except Exception as e:
                    print(f"Errore durante la processazione della patch {patch_name}: {e}")

    print(f"Creazione lista immagini per '{split}' completata. File salvato in: {output_file}")


# Crea gli elenchi
base_dir = "/mnt/d/objects365"
threads = 16
create_image_list(base_dir, "train", f"{base_dir}/train_images.txt", num_threads=threads)
create_image_list(base_dir, "val", f"{base_dir}/val_images.txt", num_threads=threads)
create_image_list(base_dir, "test", f"{base_dir}/test_images.txt", num_threads=threads)

Qui un estratto del file YAML:
```yaml
path: /mnt/d/objects365
train: train_images.txt  # File con elenco immagini di training
val: val_images.txt      # File con elenco immagini di validation
test: test_images.txt    # File con elenco immagini di test
```

Siccome mi sono accorto che nelle etichette spesso compare la classe 365 che non è mappata nel file YAML e siccome YOLO non ha la classe `background` per identificare tutto ciò che non è mappabile dal modello, ho deciso di mappare la classe 365 delle etichette come classe `Unknown`.

```yaml
# Classes
names:
  0: Person
  1: Sneakers
  2: Chair
  3: Other Shoes
  ...
  361: Lipstick
  362: Cosmetics Mirror
  363: Curling
  364: Table Tennis
  365: Unknown
```

Il dataset è molto grande, iniziamo con un subsampling che poi tratteremo come sopra ossia creando un TXT con le coordinate delle immagini e delle labels. Qui sotto un estratto:

In [28]:
import os
from pathlib import Path
import random

def create_subsample_image_list(base_dir, output_dir, train_patches=2, val_patches=1):
    """
    Crea un subsample dei file di coordinate (file .txt) per train e val selezionando un numero limitato di patch.
    
    Args:
        base_dir (str): Percorso della directory principale del dataset originale.
        output_dir (str): Percorso della directory dove salvare i file di output.
        train_patches (int): Numero di patch da includere nel train set.
        val_patches (int): Numero di patch da includere nel val set.
    """
    base_dir = Path(base_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Directory delle immagini
    train_images_dir = base_dir / "images" / "train"
    val_images_dir = base_dir / "images" / "val"

    # File di output
    train_output_file = output_dir / "train_images.txt"
    val_output_file = output_dir / "val_images.txt"

    # Seleziona patch casuali per train
    train_patches_list = random.sample(
        [d for d in os.listdir(train_images_dir) if (train_images_dir / d).is_dir()],
        train_patches
    )
    with open(train_output_file, "w") as f:
        for patch in train_patches_list:
            patch_path = train_images_dir / patch
            for img in patch_path.glob("*.jpg"):
                f.write(f"{img.absolute()}\n")
            print(f"Patch train aggiunta: {patch}")

    # Seleziona patch casuali per val
    val_patches_list = random.sample(
        [d for d in os.listdir(val_images_dir) if (val_images_dir / d).is_dir()],
        val_patches
    )
    with open(val_output_file, "w") as f:
        for patch in val_patches_list:
            patch_path = val_images_dir / patch
            for img in patch_path.glob("*.jpg"):
                f.write(f"{img.absolute()}\n")
            print(f"Patch val aggiunta: {patch}")

    print("Subsample dei file di coordinate creato con successo!")

# Esegui la creazione del subsample
base_dir = "/mnt/d/objects365"  # Percorso del dataset originale
output_dir = "/mnt/d/objects365_subsample_4"  # Percorso per salvare i file di coordinate
create_subsample_image_list(base_dir, output_dir, train_patches=4, val_patches=1)

Patch train aggiunta: patch37
Patch train aggiunta: patch28
Patch train aggiunta: patch22
Patch train aggiunta: patch50
Patch val aggiunta: patch29
Subsample dei file di coordinate creato con successo!


## Creazione Dataset Binario
Siccome le operazioni I/O sono un forte collo di bottiglia si può convertire il dataset in formato binario che dovrebbe essere molto più efficente. Purtroppo anche il subsampling ha i suoi limiti visto che per elaborare solo due patch su cinquanta ci si impiegano oltre 4 ore con il mio PC:
- Nvidia RTX 4090 (24GB VRAM)
- AMD Ryzen 9 5900X (12/24)
- 64GB RAM

In [ ]:
import os
import cv2
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Resize coerente con YOLO
img_size = (640, 640)

def process_file(fname, orig_images, orig_labels, cached_images, cached_labels):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        return
    stem = os.path.splitext(fname)[0]

    # Immagine
    img = cv2.imread(os.path.join(orig_images, fname))
    if img is None:
        print(f"❌ Impossibile leggere l'immagine: {os.path.join(orig_images, fname)}")
        return
    img = cv2.resize(img, img_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(cached_images, stem + ".jpg"), img)

    # Copia label
    orig_label_path = os.path.join(orig_labels, stem + ".txt")
    new_label_path = os.path.join(cached_labels, stem + ".txt")
    if os.path.exists(orig_label_path):
        with open(orig_label_path, "r") as f_in, open(new_label_path, "w") as f_out:
            f_out.write(f_in.read())

for fase in ["train", "val", "test"]:
    # Cartelle originali e pre-elaborate
    orig_images = f"/mnt/d/objects365/images/{fase}"
    orig_labels = f"/mnt/d/objects365/labels/{fase}"
    cached_images = f"/mnt/e/yolo-fast/images/{fase}"
    cached_labels = f"/mnt/e/yolo-fast/labels/{fase}"
    os.makedirs(cached_images, exist_ok=True)
    os.makedirs(cached_labels, exist_ok=True)

    # Lista dei file (ricorsiva)
    file_list = []
    for root, _, files in os.walk(orig_images):  # Ricerca ricorsiva
        for file in files:
            file_list.append((root, file))  # Salva il percorso completo e il nome del file

    # Elaborazione multithread
    with ThreadPoolExecutor(max_workers=20) as executor:
        list(tqdm(
            executor.map(
                lambda args: process_file(
                    args[1],  # Nome del file
                    args[0],  # Percorso della sottocartella
                    orig_labels, 
                    cached_images, 
                    cached_labels
                ),
                file_list
            ),
            total=len(file_list)
        ))

 28%|██▊       | 486620/1742341 [33:00<1:53:48, 183.89it/s]

## ✅ Step 2 – Addestramento del Modello

Il modello di YOLO scelto sarà un *pretrained* su cui verranno impostati parametri nel tentativo di ottimizzare fin da subito.


In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

## Augmentation Parameters
Gli aumenti dei dati sono tecniche utilizzate per migliorare la robustezza e la generalizzazione del modello. Ecco i parametri configurabili:

- **hsv_h (Hue)**: Regola la tonalità dei colori per rendere il modello invariante alle variazioni di colore.
- **hsv_s (Saturation)**: Modifica la saturazione per simulare diverse condizioni di illuminazione.
- **hsv_v (Brightness)**: Cambia la luminosità per gestire immagini con diverse esposizioni.
- **degrees**: Applica rotazioni casuali per migliorare l'invarianza al punto di vista.
- **translate**: Introduce traslazioni casuali per rendere il modello robusto a oggetti in posizioni diverse.
- **scale**: Ridimensiona gli oggetti per gestire variazioni di dimensioni.
- **shear**: Applica tagli (shearing) per simulare distorsioni geometriche (impostato a 0 per evitare distorsioni estreme).
- **fliplr**: Effettua un flip orizzontale casuale per aumentare la varietà dei dati.
- **mosaic**: Combina più immagini in un'unica immagine per migliorare il rilevamento di piccoli oggetti e il contesto.
- **bgr**: Scambia i canali BGR per gestire diversi formati di immagine.

In [ ]:
data = "data/objects365.yaml"  # The YAML file with paths and classes
data_cached = "data/objects365_cached.yaml"  # The YAML file with paths and classes
ephocs = 10
imgsz = 640
device = "cuda"  # set device to 0 (cuda) for GPU or 'cpu' for CPU or a numeric list '0,1,..,n' for multiple (n) GPUs
patience = 5  # early stopping patience
batch:int = -1  # batch size | if integer rappresents the number of images, if -1 use 60% of VRAM
optimizer = "AdamW"
workers = 20  # number of workers for dataloader
save = True

In [31]:


# Configura gli aumenti dei dati
augmentation_params = {
    "hsv_h": 0.015,  # Regolazione della tonalità
    "hsv_s": 0.7,    # Regolazione della saturazione
    "hsv_v": 0.4,    # Regolazione della luminosità
    "degrees": 5.0,  # Rotazione
    "translate": 0.1,  # Traslazione
    "scale": 0.5,    # Ridimensionamento
    "shear": 0.0,    # Taglio (impostato a 0 per evitare distorsioni estreme)
    "fliplr": 0.5,   # Flip orizzontale casuale
    "mosaic": 1.0,   # Aumento a mosaico
    "bgr": 1.0       # Scambio dei canali BGR
}

# Addestra il modello con gli aumenti dei dati
results = model.train(
    data=data,  # Percorso del file YAML
    epochs=ephocs,               # Numero di epoche
    imgsz=imgsz,               # Dimensione delle immagini
    batch=batch,                # Batch size
    device=device,                # GPU (0) o CPU ('cpu')
    patience=patience,              # Early stopping
    optimizer=optimizer,       # Ottimizzatore
    save=save,          # Salva i checkpoint    
    workers=workers,        # Numero di worker per il dataloader               
    **augmentation_params    # Parametri di aumento dei dati
)
# Crea la cartella 'results' se non esiste
os.makedirs("results", exist_ok=True)

# Salva i risultati in un file di testo
with open("results/training_results.txt", "w") as f:
    f.write(str(results))
    print("Risultati del training salvati in 'results/training_results.txt'.")
# Salva il modello addestrato
model.save("results/best_model.pt")

New https://pypi.org/project/ultralytics/8.3.122 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.116 🚀 Python-3.12.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=objects365.yaml, epochs=10, time=None, patience=5, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=20, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning /mnt/d/objects365/labels/train/patch22... 137492 images, 0 backgrounds, 1 corrupt: 100%|██████████| 137492/137492 [04:13<00:00, 542.22it/s]

train: /mnt/d/objects365/images/train/patch37/objects365_v2_01766463.jpg: ignoring corrupt image/label: Label class 365 exceeds dataset class count 365. Possible class labels are 0-364


train: New cache created: /mnt/d/objects365/labels/train/patch22.cache
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4090) 23.99G total, 5.48G reserved, 0.17G allocated, 18.33G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2735895       7.216         1.063          34.9         60.71        (1, 3, 640, 640)                    list
     2735895       14.43         1.095         40.52          52.1        (2, 3, 640, 640)                    list
     2735895       28.86         1.332         77.47          51.5        (4, 3, 640, 640)                    list
     2735895       57.73         1.942         53.96         84.99        (8, 3, 640, 640)                    list
     2735895       115.5         3.228           820         80.82       (16, 3, 640, 640)                    list
     2735895       230.9         6.027         1

train: Scanning /mnt/d/objects365/labels/train/patch22.cache... 137492 images, 0 backgrounds, 1 corrupt: 100%|██████████| 137492/137492 [00:00<?, ?it/s]

...

                   all       1707      25342     0.0149      0.106     0.0253     0.0174

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      13.6G      1.213      1.406      1.254        195        640: 100%|██████████| 2292/2292 [09:42<00:00,  3.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:48<00:00,  3.23s/it]


                   all       1707      25342     0.0151      0.108      0.026     0.0178

10 epochs completed in 1.741 hours.
Optimizer stripped from /home/simon/GitHub/upo-machine-learning/runs/detect/train10/weights/last.pt, 5.8MB
Optimizer stripped from /home/simon/GitHub/upo-machine-learning/runs/detect/train10/weights/best.pt, 5.8MB

Validating /home/simon/GitHub/upo-machine-learning/runs/detect/train10/weights/best.pt...
Ultralytics 8.3.116 🚀 Python-3.12.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO11n summary (fused): 100 layers, 2,727,883 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:14<00:00,  1.02it/s]


                   all       1707      25342     0.0151      0.108      0.026     0.0179
              Sneakers        779       1809    0.00287     0.0243    0.00147   0.000859
                 Chair         57        103   4.64e-05     0.0194   2.37e-05   1.31e-05
           Other Shoes        167        282   0.000192     0.0106   9.74e-05   5.84e-05
                   Hat        122        238   8.34e-05     0.0042    4.2e-05    4.2e-06
                   Car        134        193   0.000316     0.0363   0.000165   6.09e-05
                  Lamp         99        200          0          0          0          0
               Glasses        140        284          0          0          0          0
                Bottle        151        166          0          0          0          0
                  Desk        230        441     0.0167       0.44     0.0574     0.0394
                   Cup        289        377     0.0286      0.459     0.0735      0.046
         Street Light

## ✅ Step 3 – Validazione del Modello

In [18]:
print(model.info())

YOLO11n summary (fused): 100 layers, 2,727,883 parameters, 0 gradients, 7.1 GFLOPs
(100, 2727883, 0, 7.081376000000001)


In [7]:
import os
import json

# Percorso del file del modello salvato
model_path = "results/best_model.pt"

# Carica il modello YOLO
model = YOLO(model_path)


# Esegui la validazione
val_results = model.val(
    data=data,
    imgsz=imgsz,
    batch=16,
    device=device,
    save=True
)

# Crea la cartella dei risultati
os.makedirs("results", exist_ok=True)

# Ottieni il dizionario con le metriche
metrics = val_results.results_dict


filename = "results/validation_results_subest_training_4"

# Stampa tutte le chiavi disponibili per debug (facoltativo)
print("Chiavi disponibili in results_dict:")
for k in metrics:
    print(f"- {k}")

# Salvataggio in TXT
with open(f"{filename}.txt", "w") as f:
    f.write("Validation Results:\n")
    f.write(f"mAP50: {metrics.get('mAP50', 0.0):.4f}\n")
    f.write(f"mAP50-95: {metrics.get('mAP50-95', 0.0):.4f}\n")
    f.write(f"Precision: {metrics.get('precision', 0.0):.4f}\n")
    f.write(f"Recall: {metrics.get('recall', 0.0):.4f}\n")
    f.write(f"Inference Speed: {val_results.speed.get('inference', 0.0):.2f} ms/img\n")

# Salvataggio in JSON
with open(f"{filename}.json", "w") as f_json:
    json.dump(metrics, f_json, indent=4)

print(f"✅ Risultati salvati in '{filename}.txt' e '{filename}.json'")


Ultralytics 8.3.116 🚀 Python-3.12.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO11n summary (fused): 100 layers, 2,727,883 parameters, 0 gradients, 7.1 GFLOPs
val: Fast image access ✅ (ping: 2.3±0.2 ms, read: 39.2±17.6 MB/s, size: 145.9 KB)


val: Scanning /mnt/d/objects365/labels/val/patch29.cache... 1709 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1709/1709 [00:00<?, ?it/s]

val: /mnt/d/objects365/images/val/patch29/objects365_v2_01425654.jpg: ignoring corrupt image/label: Label class 365 exceeds dataset class count 365. Possible class labels are 0-364
val: /mnt/d/objects365/images/val/patch29/objects365_v2_01445910.jpg: ignoring corrupt image/label: Label class 365 exceeds dataset class count 365. Possible class labels are 0-364



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 107/107 [00:12<00:00,  8.56it/s]


                   all       1707      25342     0.0152      0.108     0.0258     0.0178
              Sneakers        779       1809    0.00287     0.0243    0.00148   0.000863
                 Chair         57        103   4.64e-05     0.0194   2.37e-05   1.31e-05
           Other Shoes        167        282   0.000191     0.0106   9.95e-05   6.25e-05
                   Hat        122        238   8.32e-05     0.0042   4.19e-05   4.19e-06
                   Car        134        193   0.000312     0.0363   0.000163   6.51e-05
                  Lamp         99        200          0          0          0          0
               Glasses        140        284          0          0          0          0
                Bottle        151        166          0          0          0          0
                  Desk        230        441     0.0169      0.444     0.0571     0.0394
                   Cup        289        377     0.0287      0.454     0.0758     0.0473
         Street Light